Group Members: Sowmya Renukuntla, Parneet Kaur, Cheyenne Edwards

OMIS 116: Applied Machine Learning

Professor Wilson Lin

# Table of Contents
1. [Previewing the Data](#preview-data)
    1. [Yield Data](#yield-data)
    2. [Temperature Data](#temperature-data)
    3. [Rainfall Data](#rainfall-data)
    4. [Pesticides Data](#pesticides-data)
2. [Merging Data](#merge-data) 
3. [Reducing Dataset](#reduce-data)

#### Data Preprocessing and Cleaning 

In [2]:
# Important Libraries for Data Analysis 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # Useful for Visualizating Missing Values or Distribuions
import seaborn as sns

In [3]:
# Loading all the datasets
yield_df = pd.read_csv('yield.csv')
rain_df = pd.read_csv('rainfall.csv')
temp_df = pd.read_csv('temp.csv')
pest_df = pd.read_csv('pesticides.csv')


## Previewing the Data
<a id="preview-data"></a>

Here are some reasons why we are previewing the data first even though there are four datasets at this moment: 
- Understand the structure (e.g. row and column count, data types, name formatting)
- Identifying Missing Values 
- Identifying Duplicate Rows/Columns 
- Are there any outliers in the data?
- Do we need to tranform the data in any way?

In [4]:
print("Yield Data:")
print(yield_df.head())

print("\nTemperature Data:")
print(temp_df.head())

print("\nRainfall Data:")
print(rain_df.head())

print("\nPesticides Data:")
print(pest_df.head())

Yield Data:
  Domain Code Domain  Area Code         Area  Element Code Element  Item Code  \
0          QC  Crops          2  Afghanistan          5419   Yield         56   
1          QC  Crops          2  Afghanistan          5419   Yield         56   
2          QC  Crops          2  Afghanistan          5419   Yield         56   
3          QC  Crops          2  Afghanistan          5419   Yield         56   
4          QC  Crops          2  Afghanistan          5419   Yield         56   

    Item  Year Code  Year   Unit  Value  
0  Maize       1961  1961  hg/ha  14000  
1  Maize       1962  1962  hg/ha  14000  
2  Maize       1963  1963  hg/ha  14260  
3  Maize       1964  1964  hg/ha  14257  
4  Maize       1965  1965  hg/ha  14400  

Temperature Data:
   year        country  avg_temp
0  1849  Côte D'Ivoire     25.58
1  1850  Côte D'Ivoire     25.52
2  1851  Côte D'Ivoire     25.67
3  1852  Côte D'Ivoire       NaN
4  1853  Côte D'Ivoire       NaN

Rainfall Data:
          Area  

## 1. Yield Data
<a id="yield-data"></a>

In [5]:
yield_df.describe()

,Area Code,Element Code,Item Code,Year Code,Year,Value
count,56717.000000,56717.0,56717.000000,56717.000000,56717.000000,56717.000000
mean,125.650422,5419.0,111.611651,1989.669570,1989.669570,62094.660084
std,75.120195,0.0,101.278435,16.133198,16.133198,67835.932856
min,1.000000,5419.0,15.000000,1961.000000,1961.000000,0.000000
25%,58.000000,5419.0,56.000000,1976.000000,1976.000000,15680.000000
50%,122.000000,5419.0,116.000000,1991.000000,1991.000000,36744.000000
75%,184.000000,5419.0,125.000000,2004.000000,2004.000000,86213.000000
max,351.000000,5419.0,489.000000,2016.000000,2016.000000,1000000.000000


In [6]:
yield_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56717 entries, 0 to 56716
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Domain Code   56717 non-null  object
 1   Domain        56717 non-null  object
 2   Area Code     56717 non-null  int64 
 3   Area          56717 non-null  object
 4   Element Code  56717 non-null  int64 
 5   Element       56717 non-null  object
 6   Item Code     56717 non-null  int64 
 7   Item          56717 non-null  object
 8   Year Code     56717 non-null  int64 
 9   Year          56717 non-null  int64 
 10  Unit          56717 non-null  object
 11  Value         56717 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 5.2+ MB


In [7]:
# Check to see how many unique areas (countries) are in the dataset
len(yield_df['Area'].unique())

212

In [8]:
# Check to see how many unique years are in the dataset
len(yield_df['Year'].unique())

56

We want to identify columns we only unique value only because these kinds of columns generally provide no new/useful information.

In [9]:
checkForConstant = [col for col in yield_df.columns if yield_df[col].nunique() == 1]

print("Columns with a single unique value:", checkForConstant)

Columns with a single unique value: ['Domain Code', 'Domain', 'Element Code', 'Element', 'Unit']


Filter-Based Feature Selection 
- Goal: Remove redundant features (e.g., columns with constant values).

In [10]:
yield_df.head() 
# We can notice that 'Domain', 'Element', 'Year Code', 'Area Code', 'Item Code', 'Domain Code' are irrelevant columns and have costant value in each instance

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


In [11]:
# Checking Unique Value Count for each Column
for c in yield_df.columns:
    print(c, yield_df[c].nunique())

Domain Code 1
Domain 1
Area Code 212


Area 212
Element Code 1
Element 1
Item Code 10
Item 10
Year Code 56
Year 56
Unit 1
Value 36815


In [12]:
# Year Code is same as Year so it is dropped 
# Item Code is just giving a code for each Item which is not necesssary 
# The Unit column provides context. If we remove it, someone analyzing the data later might not know what the numbers represent.
yield_df = yield_df.drop(['Domain', 'Element', 'Year Code', 'Area Code', 'Item Code', 'Domain Code', 'Element Code'], 
                            axis=1)

In [13]:
yield_df.columns

Index(['Area', 'Item', 'Year', 'Unit', 'Value'], dtype='object')

In [14]:
# No missing values in the dataset
yield_df.isnull().sum()

Area     0
Item     0
Year     0
Unit     0
Value    0
dtype: int64

In [15]:
# Code Cell to Check for Duplicate Rows in the Dataset
yield_duplicate_rows = yield_df.duplicated()
yield_duplicate_rows.sum()

0

## 2. Temeprature Data
<a id=temperature-data></a>

In [16]:
temp_df.describe()

,year,avg_temp
count,71311.000000,68764.000000
mean,1905.799007,16.183876
std,67.102099,7.592960
min,1743.000000,-14.350000
25%,1858.000000,9.750000
50%,1910.000000,16.140000
75%,1962.000000,23.762500
max,2013.000000,30.730000


In [17]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71311 entries, 0 to 71310
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      71311 non-null  int64  
 1   country   71311 non-null  object 
 2   avg_temp  68764 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [18]:
# Check to see how many unique areas (countries) are in the dataset
print(len(temp_df['country'].unique()))

137


In [19]:
# Check to see how many unique years are in the dataset
print(len(temp_df['year'].unique()))

271


In [20]:
print(temp_df.columns)

Index(['year', 'country', 'avg_temp'], dtype='object')


We will rename the columns to make them more readable and understandable. We will keep the names constant across all datasets so that when we merge them into one dataset, it is easier to understand.

In [21]:
temp_df.rename(columns = {'year': 'Year', 'country': 'Area'}, inplace = True)

In [22]:
temp_df.columns

Index(['Year', 'Area', 'avg_temp'], dtype='object')

In [23]:
# There are 2547 missing values in the dataset
temp_df.isnull().sum()

Year           0
Area           0
avg_temp    2547
dtype: int64

Extracting Insights: 
1. Why is the data missing? Are missing values random or systematic?
- Is it MCAR? (Missing Completely at Random) - This is what we hope for because it means that the missing data is not related to any other data in the dataset.
- Is it MAR? (Missing at Random) - This means that the missing average temperature values only occur in specific regions. We would have to impute for region instead of global average. 
- Is it MNAR? (Missing Not at Random) - This means that the missing data could bias the model toward high-yield regions. This would require external data, but we do not have the ability to do so.

In [24]:
temp_df[temp_df['avg_temp'].isna()]['Area'].value_counts()

United States    381
India            236
Pakistan         168
Indonesia        167
Brazil           115
                ... 
Afghanistan        2
Lebanon            2
Iran               1
Iraq               1
Jamaica            1
Name: Area, Length: 113, dtype: int64

Question: Do any of the countries that have missing temperature values (avg_temp) also have at least some non-null values?

In [25]:
countries_temp_missing = temp_df[temp_df['avg_temp'].isna()]['Area'].unique()
countries_temp_missing

array(["Côte D'Ivoire", 'United Arab Emirates', 'Nigeria', 'Ghana',
       'India', 'United States', 'Kazakhstan', 'Netherlands', 'Turkey',
       'China', 'Madagascar', 'Eritrea', 'Mali', 'Indonesia', 'Thailand',
       'Central African Republic', 'Spain', 'Venezuela', 'Colombia',
       'Lebanon', 'United Kingdom', 'Serbia', 'Brazil', 'Germany',
       'Switzerland', 'Guinea Bissau', 'Slovakia', 'Congo', 'Belgium',
       'Romania', 'Hungary', 'Burundi', 'Morocco', 'Russia', 'Moldova',
       'Sri Lanka', 'Guinea', 'Denmark', 'Senegal', 'Tanzania',
       'Bangladesh', 'Qatar', 'Cameroon', 'Ireland', 'Pakistan',
       'Sierra Leone', 'Guyana', 'Mexico', 'Guatemala', 'Ecuador',
       'Zimbabwe', 'Finland', 'Vietnam', 'Iraq', 'Sudan', 'Afghanistan',
       'Uganda', 'Nepal', 'Ukraine', 'Rwanda', 'Canada', 'Jamaica',
       'Congo (Democratic Republic Of The)', 'Malaysia', 'Gabon',
       'Malawi', 'Peru', 'Slovenia', 'Angola', 'Zambia',
       'Equatorial Guinea', 'Nicaragua', 'Bahra

In [26]:
# This is just used for visualization purposes
missing_counts_after_check = temp_df[temp_df['Area'].isin(countries_temp_missing)].groupby('Area')['avg_temp'].apply(lambda x: x.isna().sum())
missing_counts_after_check.to_csv("missing_temperature_values_per_country.csv", index=True)

Handling Missing Values Approach: 
- If a country has less than or equal to 30% missing values, we will use interpolation because there are enough existing data points to estimate the missing values reasonably.
- If a country has more than 70% missing values, we will drop those records since there isn’t enough information to generate reliable interpolations.
- If a country has between 30% and 70% missing values, we will use the KNN method to impute the missing values.

We got this idea from the following article: https://medium.com/@hhuseyincosgun/dealing-with-missing-data-from-zero-to-advanced-4fb734ee5998

In [27]:
from sklearn.impute import KNNImputer

In [28]:
# Percentage of Missing Values 
missing_values_proportion = temp_df.groupby('Area')['avg_temp'].apply(lambda x: x.isnull().mean() * 100)

# Check for <= 30% missing values
countries_for_interpolation = missing_values_proportion[missing_values_proportion <= 30].index

# Check for countries with > 70% missing values 
drop_countries = missing_values_proportion[missing_values_proportion > 70].index

# Check countries with missing values between 30% and 70%
knn_countries = missing_values_proportion[(missing_values_proportion > 30) & (missing_values_proportion <= 70)].index

# Interpolate 
for country in countries_for_interpolation:
    temp_df.loc[temp_df['Area'] == country, 'avg_temp'] = temp_df.loc[temp_df['Area'] == country, 'avg_temp'].interpolate()

# KNN Imputation
for country in knn_countries:
    # Small n_neighbors to avoid averaging over too many
    imputer = KNNImputer(n_neighbors = 4)
    temp_df.loc[temp_df['Area'] == country, 'avg_temp'] = imputer.fit_transform(temp_df[temp_df['Area'] == country][['avg_temp']])

# Drop
for country in drop_countries:
    temp_df = temp_df[temp_df['Area'] != country]

In [29]:
temp_df.isnull().sum()

Year        0
Area        0
avg_temp    0
dtype: int64

In [30]:
duplicate_years = temp_df[temp_df.duplicated(subset=['Year'], keep=False)]
duplicate_years

,Year,Area,avg_temp
0,1849,Côte D'Ivoire,25.580
1,1850,Côte D'Ivoire,25.520
2,1851,Côte D'Ivoire,25.670
3,1852,Côte D'Ivoire,25.792
4,1853,Côte D'Ivoire,25.914
...,...,...,...
71306,2009,Mexico,21.760
71307,2010,Mexico,20.900
71308,2011,Mexico,21.550
71309,2012,Mexico,21.520


In [31]:
temp_df[(temp_df['Year'] == 1849) & (temp_df['Area'] == "Côte D'Ivoire")]

,Year,Area,avg_temp
0,1849,Côte D'Ivoire,25.58
70465,1849,Côte D'Ivoire,25.00


## 3. Rainfall Data
<a id=rainfall-data></a>


In [32]:
rain_df.describe()

,Year
count,6727.000000
mean,2001.354839
std,9.530114
min,1985.000000
25%,1993.000000
50%,2001.000000
75%,2010.000000
max,2017.000000


In [33]:
rain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6727 entries, 0 to 6726
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0    Area                          6727 non-null   object
 1   Year                           6727 non-null   int64 
 2   average_rain_fall_mm_per_year  5953 non-null   object
dtypes: int64(1), object(2)
memory usage: 157.8+ KB


In [34]:
rain_df.columns

Index([' Area', 'Year', 'average_rain_fall_mm_per_year'], dtype='object')

We can notice that the Area column has a whitespace at the beginning of the column name. We will remove that whitespace.

In [35]:
rain_df.columns = rain_df.columns.str.strip()

In [36]:
# Check to see how many unique areas (countries) are in the dataset
print(len(rain_df['Area'].unique()))

217


In [37]:
# Check to see how many unique years are in the dataset
print(len(rain_df['Year'].unique()))

31


In [38]:
rain_df.rename(columns = {'average_rain_fall_mm_per_year': 'Average_Rainfall_Per_Year'}, inplace = True)

In [39]:
rain_df.columns

Index(['Area', 'Year', 'Average_Rainfall_Per_Year'], dtype='object')

In [40]:
# There are 774 missing values in the dataset
rain_df.isnull().sum()

Area                           0
Year                           0
Average_Rainfall_Per_Year    774
dtype: int64

Extracting Insights: 
1. Why is the data missing? Are missing values random or systematic?
- Is it MCAR? (Missing Completely at Random) - This is what we hope for because it means that the missing data is not related to any other data in the dataset.
- Is it MAR? (Missing at Random) - This means that the missing rainfall values only occur in specific regions. We would have to impute for region instead of global average. 
- Is it MNAR? (Missing Not at Random) - This means that the missing data could bias the model toward high-yield regions. This would require external data, but we do not have the ability to do so.

In [41]:
rain_df[rain_df['Average_Rainfall_Per_Year'].isna()]['Area'].value_counts()

American Samoa               31
Aruba                        31
Turks and Caicos Islands     31
Tonga                        31
St. Martin (French part)     31
Sint Maarten (Dutch part)    31
San Marino                   31
Northern Mariana Islands     31
New Caledonia                31
Macao SAR, China             31
Kosovo                       31
Isle of Man                  31
Hong Kong SAR, China         31
Guam                         31
Greenland                    31
Gibraltar                    31
French Polynesia             31
Faroe Islands                31
Curacao                      31
Channel Islands              31
Cayman Islands               31
British Virgin Islands       31
Bermuda                      31
Virgin Islands (U.S.)        31
Monaco                       30
Name: Area, dtype: int64

Question: Do any of the countries above that have missing values also have at least some non-null values?

In [42]:
missing_countries = rain_df[rain_df['Average_Rainfall_Per_Year'].isna()]['Area'].unique()

In [43]:
check_for_non_null = rain_df[(rain_df['Area'].isin(missing_countries)) & (rain_df['Average_Rainfall_Per_Year'].notnull())]
print(check_for_non_null[['Area', 'Average_Rainfall_Per_Year']].drop_duplicates())

        Area Average_Rainfall_Per_Year
4061  Monaco                        ..


Only Monaco has a non-null value which is actually a blank space. We will consider this as a missing value and drop the row safely alongside the countries that have missing values.

In [44]:
rain_df = rain_df.dropna(subset=['Average_Rainfall_Per_Year'])

In [45]:
rain_df.isna().sum()

Area                         0
Year                         0
Average_Rainfall_Per_Year    0
dtype: int64

In [46]:
# There are no duplicate rows in the dataset
rain_duplicate_rows = rain_df.duplicated(keep=False)
rain_duplicate_rows.sum()

0

## 4. Pesticide Data
<a id=pesticides-data></a>

In [47]:
pest_df.describe()

,Year,Value
count,4349.000000,4.349000e+03
mean,2003.138883,2.030334e+04
std,7.728044,1.177362e+05
min,1990.000000,0.000000e+00
25%,1996.000000,9.300000e+01
50%,2003.000000,1.137560e+03
75%,2010.000000,7.869000e+03
max,2016.000000,1.807000e+06


In [48]:
pest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4349 entries, 0 to 4348
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Domain   4349 non-null   object 
 1   Area     4349 non-null   object 
 2   Element  4349 non-null   object 
 3   Item     4349 non-null   object 
 4   Year     4349 non-null   int64  
 5   Unit     4349 non-null   object 
 6   Value    4349 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 238.0+ KB


In [49]:
pest_df.columns

Index(['Domain', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value'], dtype='object')

In [50]:
# Check to see how many unique areas (countries) are in the dataset
print(len(pest_df['Year'].unique()))

27


In [51]:
# Check to see how many unique years are in the dataset
print(len(pest_df['Area'].unique()))

168


In [52]:
checkForConstant2 = [col for col in pest_df.columns if pest_df[col].nunique() == 1]

print("Columns with a single unique value:", checkForConstant2)

Columns with a single unique value: ['Domain', 'Element', 'Item', 'Unit']


In [53]:
# Code Cell for Dropping Columns with a Single Unique Value

In [54]:
# There are no missing values in the dataset
pest_df.isnull().sum()

Domain     0
Area       0
Element    0
Item       0
Year       0
Unit       0
Value      0
dtype: int64

In [55]:
# Code Cell to Check for Duplicate Rows in the Dataset
pest_duplicate_rows = pest_df.duplicated()
pest_duplicate_rows.sum()

0

## Merging Data
<a id="merge-data"></a>

Merging all the data provides a single dataset that can be used for training and testing the model. This helps because many of the datasets had complementary information.

In [57]:
# First, merge yield_df and rainfall_df on Year and Area
merged_df = pd.merge(yield_df, rain_df, on=['Year', 'Area'], how='inner')

# Second, merge the resulting dataframe with pesticides_df
merged_df = pd.merge(merged_df, pest_df, on=['Year', 'Area'], how='inner')

# Finally, merge the resulting dataframe with temp_df
merged_df = pd.merge(merged_df, temp_df, on=['Year', 'Area'], how='inner')

In [58]:
# Display the first few rows of the merged dataframe
merged_df.head()

,Area,Item_x,Year,Unit_x,Value_x,Average_Rainfall_Per_Year,Domain,Element,Item_y,Unit_y,Value_y,avg_temp
0,Albania,Maize,1990,hg/ha,36613,1485,Pesticides Use,Use,Pesticides (total),tonnes of active ingredients,121.0,16.37
1,Albania,Potatoes,1990,hg/ha,66667,1485,Pesticides Use,Use,Pesticides (total),tonnes of active ingredients,121.0,16.37
2,Albania,"Rice, paddy",1990,hg/ha,23333,1485,Pesticides Use,Use,Pesticides (total),tonnes of active ingredients,121.0,16.37
3,Albania,Sorghum,1990,hg/ha,12500,1485,Pesticides Use,Use,Pesticides (total),tonnes of active ingredients,121.0,16.37
4,Albania,Soybeans,1990,hg/ha,7000,1485,Pesticides Use,Use,Pesticides (total),tonnes of active ingredients,121.0,16.37


In [59]:
merged_df.rename(columns = {
    'Value_x': 'Value_Yield',  # Renaming Value_x to Value_Yield for better understanding
    'Value_y': 'Value_Pesticide'  # Renaming Value_y to Value_Pesticide for better understanding
}, inplace = True)

In [60]:
# Final check for missing values
merged_df.isnull().sum()

Area                         0
Item_x                       0
Year                         0
Unit_x                       0
Value_Yield                  0
Average_Rainfall_Per_Year    0
Domain                       0
Element                      0
Item_y                       0
Unit_y                       0
Value_Pesticide              0
avg_temp                     0
dtype: int64

In [61]:
# Check to see how many unique areas (countries) are in the dataset
len(merged_df['Year'].unique())

23

In [62]:
# Check to see how many unique years are in the dataset
len(merged_df['Area'].unique())

101

In [63]:
merged_df.shape

(28248, 12)

## Reducing the Data
<a id="reduce-data"></a>

Here, we are getting rid of years before 1990. Starting at 1990 will allow us to have data from all of the datasets. 
- Other Reasons - Data before 1990 may be less relevant for modern climate analysis.
- Many environmental patterns have changed due to climate change, industrialization, and policy shifts.
- It also may improve model performance because this may reduce noise, fasten training time, and reduce memory usage.